In [2]:
import tensorflow as tf
import efficientnet.keras as efn 
import numpy as np
import os
import keras
import cv2

from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Flatten, AveragePooling2D, Input, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import SGD,Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint


from livelossplot import PlotLossesKeras

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, classification_report

from tensorflow.compat.v1.keras.backend import set_session

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5



In [3]:
labels_dict_fer = {
    "angry": 0,
    "disgust": 1,
    "fear": 2,
    "happy": 3,
    "sad": 4,
    "surprise": 5,
    "neutral": 6
}

In [4]:
path = "/home/dl/1_study/0_BSU/master_thesis/dataset/emotions/base/"

dataset_1 = "fer2013"
dataset_1_type = "crop"
dataset_1_upscale = True

dataset_2 = "jaffedbase"
dataset_2_type = "crop"
dataset_2_upscale = False

dataset_3 = "KDEF_and_AKDEF"
dataset_3_type = "crop"
dataset_3_upscale = False

In [5]:
ROWS = 128
COLS = 128
CHANNELS = 1
NUM_CLASSES = 7

# dataset

In [6]:
# fer2013 train
X_train_fer2013 = []
y_train_fer2013 = []

for emotion in os.listdir(os.path.join(path, dataset_1, "upscale", dataset_1_type, "train")):
    for img_name in os.listdir(os.path.join(path, dataset_1, "upscale", dataset_1_type, "train", emotion)):
        X_train_fer2013.append(os.path.join(path, dataset_1, "upscale", dataset_1_type, "train", emotion, img_name))
        y_train_fer2013.append(labels_dict_fer[emotion])

X_train_fer2013 = np.array(X_train_fer2013)
y_train_fer2013_one_hot = to_categorical(y_train_fer2013, num_classes=7)

print(X_train_fer2013.shape)
print(y_train_fer2013_one_hot.shape)

(27952,)
(27952, 7)


In [7]:
# fer2013 val
X_val_fer2013 = []
y_val_fer2013 = []

for folder_name in ["public_test"]:
    for emotion in os.listdir(os.path.join(path, dataset_1, "upscale", dataset_1_type, folder_name)):
        for img_name in os.listdir(os.path.join(path, dataset_1, "upscale", dataset_1_type, folder_name, emotion)):
            X_val_fer2013.append(os.path.join(path, dataset_1, "upscale", dataset_1_type, folder_name, emotion, img_name))
            y_val_fer2013.append(labels_dict_fer[emotion])

X_val_fer2013 = np.array(X_val_fer2013)
y_val_fer2013_one_hot = to_categorical(y_val_fer2013, num_classes=NUM_CLASSES)

print(X_val_fer2013.shape)
print(y_val_fer2013_one_hot.shape)

# fer2013 test
X_test_fer2013 = []
y_test_fer2013 = []
for folder_name in ["private_test"]:
    for emotion in os.listdir(os.path.join(path, dataset_1, "upscale", dataset_1_type, folder_name)):
        for img_name in os.listdir(os.path.join(path, dataset_1, "upscale", dataset_1_type, folder_name, emotion)):
            X_test_fer2013.append(os.path.join(path, dataset_1, "upscale", dataset_1_type, folder_name, emotion, img_name))
            y_test_fer2013.append(labels_dict_fer[emotion])

X_test_fer2013 = np.array(X_test_fer2013)
y_test_fer2013_one_hot = to_categorical(y_test_fer2013, num_classes=NUM_CLASSES)

print(X_test_fer2013.shape)
print(y_test_fer2013_one_hot.shape)

(3486,)
(3486, 7)
(3499,)
(3499, 7)


In [8]:
# jaffedbase
X_jaffedbase = [] 
y_jaffedbase = []

for emotion in os.listdir(os.path.join(path, dataset_2, "preprocess", dataset_2_type)):
    for img_name in os.listdir(os.path.join(path, dataset_2, "preprocess", dataset_2_type, emotion)):
        X_jaffedbase.append(os.path.join(path, dataset_2, "preprocess", dataset_2_type, emotion, img_name))
        y_jaffedbase.append(labels_dict_fer[emotion])
        
X_jaffedbase = np.array(X_jaffedbase)
y_jaffedbase_one_hot = to_categorical(y_jaffedbase, num_classes=NUM_CLASSES)

X_train_jaffedbase, X_test_jaffedbase, y_train_jaffedbase, y_test_jaffedbase = \
train_test_split(X_jaffedbase, y_jaffedbase_one_hot, test_size=0.2, random_state=42, stratify=y_jaffedbase)

print(X_train_jaffedbase.shape)
print(y_train_jaffedbase.shape)
print(X_test_jaffedbase.shape)
print(y_test_jaffedbase.shape)

(78,)
(78, 7)
(20,)
(20, 7)


In [9]:
# KDEF_and_AKDEF
X_kdef = [] 
y_kdef = []

for emotion in os.listdir(os.path.join(path, dataset_3, "preprocess", dataset_3_type)):
    for img_name in os.listdir(os.path.join(path, dataset_3, "preprocess", dataset_3_type, emotion)):
        X_kdef.append(os.path.join(path, dataset_3, "preprocess", dataset_3_type, emotion, img_name))
        y_kdef.append(labels_dict_fer[emotion])

X_kdef = np.array(X_kdef)
y_kdef_one_hot = to_categorical(y_kdef, num_classes=NUM_CLASSES)

X_train_kdef, X_test_kdef, y_train_kdef, y_test_kdef = \
train_test_split(X_kdef, y_kdef_one_hot, test_size=0.2, random_state=42, stratify=y_kdef)

print(X_train_kdef.shape)
print(y_train_kdef.shape)
print(X_test_kdef.shape)
print(y_test_kdef.shape)

(3912,)
(3912, 7)
(978,)
(978, 7)


In [10]:
X_train, y_train = np.concatenate([X_train_fer2013, X_train_jaffedbase, X_train_kdef]), \
np.concatenate([y_train_fer2013_one_hot, y_train_jaffedbase, y_train_kdef])
X_val, y_val = X_val_fer2013, y_val_fer2013
X_test, y_test = np.concatenate([X_test_fer2013, X_test_jaffedbase, X_test_kdef]), \
np.concatenate([y_test_fer2013_one_hot, y_test_jaffedbase, y_test_kdef])

In [11]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(31942,)
(3486,)
(4497,)


# augmentation

In [12]:
# from albumentations import Compose, HorizontalFlip, HueSaturationValue,\
#                             Blur, ShiftScaleRotate, RandomBrightness, \
#                             Rotate,IAACropAndPad, CenterCrop, Lambda, \
#                             Downscale, Crop
# import albumentations as A

In [13]:
# aug = Compose([
#         HorizontalFlip(p=0.7),
#         Blur(blur_limit=3, p=0.6),
#         RandomBrightness(limit=0.01, p=0.6),
#         Rotate(limit=10,border_mode=cv2.BORDER_CONSTANT, value=(255, 255, 255), p=0.4),
#         HueSaturationValue(hue_shift_limit=1,
#                            sat_shift_limit=1,
#                            val_shift_limit=1,
#                            p=0.6)], p=1)

# generator

In [14]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_names, labels, batch_size=32, dim=(32,32), n_channels=1,
                 n_classes=10, shuffle=True, with_aug=False):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_names = list_names
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.with_aug = with_aug
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_names) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of images
        list_names_temp = [self.list_names[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_names_temp, indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_names))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def preprocess(self, img):
        img = cv2.resize(img, self.dim)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY).astype(np.float32)
        
#         mean = [103.939, 116.778, 123.68] # bgr
    
#         mean = [123.68, 116.778, 103.939] # rgb
#         _IMAGENET_MEAN = -np.array(mean, dtype=np.float32)
#         img += _IMAGENET_MEAN
        img /= 255.

        return img

    def __data_generation(self, list_names_temp, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels), dtype=np.float32)
        y = np.empty((self.batch_size, self.n_classes), dtype=int)

        # Generate data
        for i, name, index in zip(range(self.batch_size), list_names_temp, indexes):
            image = cv2.imread(os.path.join("cifar-10-python/data/img/", name))
            image_preproc = self.preprocess(image)
            # Store sample
            if self.with_aug:
                augmented = aug(image=image_preproc) 
                aug_image = augmented['image']
                
                X[i,] = aug_image
            else:
                X[i,] = np.expand_dims(image_preproc, -1)
                
            # Store class
            y[i] = self.labels[index]

        return X, y

# models, train

In [15]:
names_train = X_train
labels_train = y_train

names_val = X_val
labels_val = y_val

# Generators
training_generator = DataGenerator(names_train, labels_train, dim=(COLS, ROWS), batch_size=16, 
                                   n_classes=NUM_CLASSES, n_channels=1, shuffle=True, with_aug=False)
validation_generator = DataGenerator(names_val, labels_val, dim=(COLS, ROWS), batch_size=16, 
                                     n_classes=NUM_CLASSES, n_channels=1, shuffle=True, with_aug=False)

In [16]:
# EfficientNet
base_model_efficient = efn.EfficientNetB0(weights=None, include_top=False, 
                                                           input_shape= (COLS, ROWS, 1))

x = base_model_efficient.output
x = GlobalAveragePooling2D()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)
preds = Dense(NUM_CLASSES, activation='softmax')(x)

model=Model(inputs=base_model_efficient.input, outputs=preds)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 64, 64, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 64, 64, 32)   128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 64, 64, 32)   0           stem_bn[0][0]                    
_______________________________________________________________________________________

In [17]:
adam = Adam(decay=1e-6)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
def step_decay(epoch):
    initial_lrate = 0.0001
    drop = 0.5
    epochs_drop = 20
    lrate = initial_lrate * np.power(drop,  
           np.floor((1+epoch)/epochs_drop))
    return lrate

save_dir = '/home/dl/1_study/0_BSU/master_thesis/weights/emotion_clf/\
KerasEffNetB0Gray_128x128_batch_16_ALL_crop_FREP_upscale/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
lrate = LearningRateScheduler(step_decay)
checkpoints = ModelCheckpoint(filepath=os.path.join(save_dir, 'weights.{epoch:02d}-{val_loss:.2f}.h5'),
                              verbose=2,period=10) #save_best_only=True,
callbacks_list = [checkpoints, lrate, PlotLossesKeras()]

In [19]:
EPOCHS = 100
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    epochs=EPOCHS,
#                     use_multiprocessing=True,
                    callbacks=callbacks_list,
                    verbose=1,
                    workers=6)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
 164/1996 [=>............................] - ETA: 1:43 - loss: 1.9049 - accuracy: 0.2222

KeyboardInterrupt: 

In [ ]:
X_pred = np.array([cv2.imread(os.path.join("cifar-10-python/test/img/", name)) for name in X_test])
y_pred = model.predict_classes(X_pred)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1:', f1_score(y_test, y_pred, average='weighted'))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))